In [2]:
!pip install -q wandb

In [55]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import pytorch_lightning as pl
import torchmetrics
import matplotlib.pyplot as plt
import wandb
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

In [62]:
train_df = pd.read_csv('./train_data.csv')
val_df = pd.read_csv('./validation_data.csv')
test_df = pd.read_csv('./test_data.csv')
feature_cols = ['Packet speed', 'Data speed', 'Packet Rate', 'Data Rate', 'Packet Size to Speed Ratio', 'Data to Packet Ratio',
                'Duration', 'Hour sin', 'Hour cos', 'Day of Week','Unique ports', 'Std packet speed', 'Max packet speed', 
                'Std data speed', 'Max data speed', 'Std packet length', 'Max packet length',
                'Std source IP count', 'Max source IP count', 'Mean time diff', 'Std time diff','pca_1','pca_2']

features_selected = ['Data to Packet Ratio', 'Packet Size to Speed Ratio', 'Packet speed', 'Data speed', 'Max packet length', 'Hour cos', 'Day of Week', 'Std packet length', 'Unique ports']
X_train = train_df[features_selected]
y_train = train_df['Type']
X_val = val_df[features_selected]
y_val = val_df['Type']
X_test = test_df[features_selected]
y_test = test_df['Type']
le = LabelEncoder()
y_test = le.fit_transform(y_test)
y_train = le.fit_transform(y_train)
y_val = le.fit_transform(y_val)
scaler = StandardScaler()

In [49]:
batch_size = 64
num_epochs = 20
lr = 0.002
hidden_dim = 64

In [64]:
def create_dataloader(X, y, batch_size):
  scaler.fit(X)
  X = scaler.transform(X).astype('float32')
  X = torch.from_numpy(X)
  y = y.astype('long')
  y = torch.from_numpy(y)
  dataset = TensorDataset(X, y)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers = 15)
  return dataloader

train_loader = create_dataloader(X_train, y_train, batch_size)
val_loader = create_dataloader(X_val, y_val, batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size)

In [44]:
class FeedForwardNet(pl.LightningModule):
    def __init__(self, num_features, hidden_dim, lr, num_classes=3, dropout_prob=0.3):
        super(FeedForwardNet, self).__init__()
        self.save_hyperparameters()
        self.lr = lr
        self.num_classes = num_classes

        self.layers = nn.Sequential(
            nn.Linear(num_features, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_prob),

            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_prob),

            nn.Linear(hidden_dim, num_classes)  # output layer (logits)
        )

        self.loss = nn.CrossEntropyLoss()
        self.acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        return self.layers(x)

    def training_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch)
        self.log_dict({"loss": loss, "acc": acc}, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch)
        self.log_dict({"val_loss": loss, "val_acc": acc}, prog_bar=False)
        return {"val_loss": loss, "val_acc": acc}

    def test_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch)
        self.log_dict({"test_loss": loss, "test_acc": acc})
        return {"test_loss": loss, "test_acc": acc}

    def _shared_eval_step(self, batch):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = self.acc(preds, y)
        return loss, acc

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)

In [45]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: baczodomonkos (baczodomonkos-budapesti-m-szaki-s-gazdas-gtudom-nyi-egyetem) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [65]:
torch.set_float32_matmul_precision('high')
model = FeedForwardNet(X_train.shape[1], hidden_dim, lr)
wandb_logger = pl.loggers.WandbLogger(project="ddos_classifier_dl", log_model="all")
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_acc", mode="max")
trainer = pl.Trainer(
    max_epochs=10,
    precision="16-mixed",
    accelerator="gpu",
    devices=1,
    logger=wandb_logger,
    callbacks=[checkpoint_callback],
)
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)
wandb.finish()

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | layers | Sequential         | 5.3 K  | train
1 | loss   | CrossEntropyLoss   | 0      | train
2 | acc    | MulticlassAccuracy | 0      | train
------------------------------------------------------
5.3 K     Trainable params
0         Non-trainable params
5.3 K     Total params
0.021     Total estimated model params size (MB)
12        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9254773855209351     │
│         test_loss         │    0.32645171880722046    │
└───────────────────────────┴───────────────────────────┘

acc,▆▇▇█▂▁▅▆▇▅▇▆▄▆▄▅█▇▄█▅▃▄▆▆▅▇▄▄▄▃▇▄█▆▅▆▆▇▆
epoch,▁▁▂▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███████
loss,▃▅▅▅▆▇▃▃▁▆▂▂▅▃▇▃▃▄▂█▄▃▃█▅▂▃▃▄▂▃▁▅▇█▃▄▅▄▂
test_acc,▁
test_loss,▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇████
val_acc,▇█▅▅▄▁▇▁▂▅
val_loss,▁▁▂▄▄█▃▅▅▅
acc,0.95312
epoch,10
loss,0.11757


In [73]:
wandb.finish()

In [66]:
all_preds = []
all_targets = []

model.eval()
with torch.no_grad():
    for batch in test_loader:
        x, y = batch
        logits = model(x)
        preds = torch.argmax(logits, dim=1)

        all_preds.append(preds.cpu())
        all_targets.append(y.cpu())

y_pred = torch.cat(all_preds).numpy()
y_true = torch.cat(all_targets).numpy()
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.34      0.47     14733
           1       0.93      1.00      0.97    394482
           2       0.62      0.20      0.30     28424

    accuracy                           0.93    437639
   macro avg       0.78      0.51      0.58    437639
weighted avg       0.91      0.93      0.91    437639



## Hiperparaméter optimalizáció

In [75]:
sweep_config = {
    'method': 'bayes',  # can be 'random', 'grid', or 'bayes'
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'batch_size': {
            'values': [32, 64, 128, 256]
        },
        'lr': {
            'distribution': 'log_uniform_values',
            'min': 1e-4,
            'max': 1e-2
        },
        'hidden_dim': {
            'values': [32, 64, 128, 256]
        },
        'dropout_prob': {
            'values': [0.1, 0.2, 0.3, 0.4, 0.5]
        },
        'num_epochs': {
            'values': [5, 8, 10, 12, 15]
        }
    }
}


In [76]:
def train_sweep():
    # Initialize a new wandb run
    wandb_run = wandb.init()
    
    # Get hyperparameters for this run
    config = wandb.config

    train_loader = create_dataloader(X_train, y_train, config.batch_size)
    val_loader = create_dataloader(X_val, y_val, config.batch_size)
    test_loader = create_dataloader(X_test, y_test, config.batch_size)
    
    # Initialize model with hyperparameters from sweep
    model = FeedForwardNet(
        num_features=X_train.shape[1],
        hidden_dim=config.hidden_dim,
        lr=config.lr,
        dropout_prob=config.dropout_prob
    )
    
    # Initialize logger and callbacks
    wandb_logger = pl.loggers.WandbLogger(project="ddos_classifier_dl", log_model="all")
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_acc", mode="max")
    
    # Initialize trainer
    trainer = pl.Trainer(
        max_epochs=config.num_epochs,
        precision="16-mixed",
        accelerator="gpu",
        devices=1,
        logger=wandb_logger,
        callbacks=[checkpoint_callback],
    )
    
    # Train and test the model
    trainer.fit(model, train_loader, val_loader)
    trainer.test(model, test_loader)

In [77]:
sweep_id = wandb.sweep(sweep_config, project="ddos_classifier_dl_sweep")
wandb.agent(sweep_id, train_sweep, count=20)

Create sweep with ID: 4qxz0kbr
Sweep URL: https://wandb.ai/baczodomonkos-budapesti-m-szaki-s-gazdas-gtudom-nyi-egyetem/ddos_classifier_dl_sweep/sweeps/4qxz0kbr


wandb: Agent Starting Run: bjl1tywq with config:
wandb: 	batch_size: 256
wandb: 	dropout_prob: 0.5
wandb: 	hidden_dim: 64
wandb: 	lr: 0.00014432875355228211
wandb: 	num_epochs: 10


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | layers | Sequential         | 5.3 K  | train
1 | loss   | CrossEntropyLoss   | 0      | train
2 | acc    | MulticlassAccuracy | 0      | train
------------------------------------------------------
5.3 K     Trainable params
0         Non-trainable params
5.3 K     Total params
0.021     Total estimated model params size (MB)
12        Modules in train mode
0         Modules in eval mode
wandb: WARNING Config item 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=15` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
wandb: WARNING Config item 'hidden_dim' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'dropout_prob' was locked by 'sweep' (ignored update).
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9221596121788025     │
│         test_loss         │    0.36698460578918457    │
└───────────────────────────┴───────────────────────────┘

acc,▄▅▆▅▁▄▄▇▅▅▆▅▅▄▇▇▅▇▂▅▅▂█▅▄▅▄▄▆▆▆▅▄▅▆▇▇▄▄▄
epoch,▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇███
loss,█▅▄▃▃▄▄▂▁▂▂▃▃▃▂▂▃▆▃▂▄▁▅▁▃▃▃▂▃▁▂▄▄▂▂▁▁▄▃▃
test_acc,▁
test_loss,▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▆▆▇▇▇▇▇▇▇▇█████
val_acc,█▇▇▆▂▂▁▁▂▂▁▁▂▂▁
val_loss,▂▁▂▂▄▄▅▅▅▅▅▅▆▆█
acc,0.9375
epoch,15
loss,0.19675


In [99]:
# Érdemes lehet majd újrafuttatni a recall-t és egyéb metrikákat számoló kódot a train_sweep függvénybe emelve.